This project aims to create a neural network capable of estimating DoA from telecommunications signals.

To do so, the training dataset consists of In-Phase and Quadrature (IQ) samples and Angle of Arrival (AoA) measures.
IQ samples consists

In [8]:
# Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras import layers, models
import os

In [9]:
# Set dataset path variable
dataset_path = os.path.join(
    os.getcwd(), "Matlab", "bluetooth_signals_dataset_2024-02-19_20-03-25.csv"
)

In [10]:
# Load dataset
data = pd.read_csv(dataset_path)

In [11]:
# Extract features (IQ samples) and labels (Angles)
X_complex_str = data.iloc[:, 1:].values.astype(str)
y_str = data.iloc[:, 0].values

# Convert string representations of complex numbers to actual complex values for features
X_complex = np.array([np.complex128(complex(val.replace('i', 'j'))) for row in X_complex_str for val in row])
X_complex = X_complex.reshape(X_complex_str.shape)

# Separate real and imaginary parts for features
X_real = np.real(X_complex)
X_imag = np.imag(X_complex)

# Combine real and imaginary parts into a single array for features
X_combined = np.stack((X_real, X_imag), axis=-1)

# Convert string representations of complex numbers to actual complex values for labels
y_complex = np.array([np.complex128(complex(val.replace('i', 'j'))) for val in y_str])

# Use only the real part for labels
y = np.real(y_complex)

# Split the combined data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Standardize the input data
scaler = StandardScaler()
X_train_flat = X_train.reshape((-1, 16))
X_test_flat = X_test.reshape((-1, 16))
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)
X_train = X_train_scaled.reshape((-1, 4, 4, 2))
X_test = X_test_scaled.reshape((-1, 4, 4, 2))

# Model architecture
model = models.Sequential()
# Tentar mudar a arquitetura: uma camada inical maior, 2 camadas convolucionais, etc.
# Tentar um dataset com menos angulos, ao invés de 361 entre -180 e 180, de 5 em 5.
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(4, 4, 2)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))  # Linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [12]:
"""
We'll use the `ModelCheckpoint` callback to regularly save checkpoints, and
the `EarlyStopping` callback to interrupt training when the validation loss
is not longer improving.
"""

path_checkpoint = "aoa_model_checkpoint.weights.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10, restore_best_weights=True)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

In [13]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks=[es_callback, modelckpt_callback])

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error on Test Set: {mae}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/20
191/217 [=========================>....] - ETA: 0s - loss: 10872.5146 - mae: 90.2660
Epoch 1: val_loss improved from inf to 10261.64258, saving model to aoa_model_checkpoint.weights.h5
217/217 [==============================] - 1s 2ms/step - loss: 10917.5762 - mae: 90.4904 - val_loss: 10261.6426 - val_mae: 87.0405
Epoch 2/20
214/217 [============================>.] - ETA: 0s - loss: 10916.3457 - mae: 90.4786
Epoch 2: val_loss did not improve from 10261.64258
217/217 [==============================] - 0s 2ms/step - loss: 10915.9062 - mae: 90.4820 - val_loss: 10262.1250 - val_mae: 87.0397
Epoch 3/20
193/217 [=========================>....] - ETA: 0s - loss: 10874.0527 - mae: 90.2126
Epoch 3: val_loss did not improve from 10261.64258
217/217 [==============================] - 0s 2ms/step - loss: 10914.0283 - mae: 90.4742 - val_loss: 10262.9092 - val_mae: 87.0421
Epoch 4/20
201/217 [==========================>...] - ETA: 0s - loss: 10926.7041 - mae: 90.6067
Epoch 4: val_loss imp

In [77]:
predictions

array([[ 0.34495637],
       [-0.40295032],
       [ 0.17824353],
       ...,
       [ 0.48004445],
       [-0.29392853],
       [ 0.5742067 ]], dtype=float32)